<a href="https://colab.research.google.com/github/Millastar/A-common-web-page-layout/blob/main/sovits_EN_training_colab_4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check graphics card
!nvidia-smi

In [ ]:
#@title Clone github repo
!git clone https://github.com/effusiveperiscope/so-vits-svc -b eff-4.0

In [ ]:
#@title Install dependencies
%cd /content/so-vits-svc
!python -m pip install --upgrade pip setuptools numpy numba
!pip install pyworld==0.3.1 praat-parselmouth
!pip install fairseq==0.12.2 librosa==0.8.1
!pip install torchcrepe tensorboardX

In [ ]:
#@title Download ContentVec model file
!wget -P hubert/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/checkpoint_best_legacy_500.pt

# Dataset preprocessing

In [ ]:
#@title Mount google drive
#@markdown Mount google drive, which will be used to hold your dataset files.
from google.colab import drive
drive.mount('/content/drive')

This processing script can preprocess multiple speakers at one time and generate multi-speaker filelists and corresponding configuration files.

First, create a folder in your Google Drive called "dataset" (no quotes).

Then, for each character you intend to train, upload the training data to Google Drive in the form of a .zip enclosing your speaker dataset (i.e. the .zip should enclose folders containing .wav files for each speaker).

Use the dialog below to load the dataset from Google Drive.



In [ ]:
#@title Load the dataset from .zip in Google Drive for preprocessing
#@markdown Name of the zip folder
DATASETNAME = "Chrysalis"  #@param {type:"string"}
#@markdown Zip path (usually do not need to change this unless you uploaded to a different directory)
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME

!unzip -d /content/so-vits-svc/dataset_raw {ZIP_NAME}.zip

In [ ]:
#@title Resample to 44.1k
!python resample.py

In [ ]:
#@title Segment training set and generate configuration files
!python preprocess_flist_config.py

In [ ]:
#@title Generate hubert and f0
!python preprocess_hubert_f0.py

In [ ]:
#@title Save preprocessed dataset files to Google Drive (for training convenience)
#Compress dataset folder
!zip -r dataset.zip /content/so-vits-svc/dataset
#@markdown Customize name of preprocessed dataset folder to avoid conflicts.
dataset_name_drive = "44k_dataset"  #@param {type:"string"}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/so-vits-svc/dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

In [ ]:
#@title Load preprocessed dataset files from Google Drive (no need to run first round)
#@markdown If you already have preprocessed dataset files on Google Drive, you can load them here instead of re-running the preprocessing steps.
back_up_name = "three_dataset"  #@param {type:"string"}
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name
!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!cp {BACK_UP_DATASET_PATH}/config.json /content/so-vits-svc/configs/config.json 
!cp {BACK_UP_DATASET_PATH}/val.txt filelists/val.txt
!cp {BACK_UP_DATASET_PATH}/train.txt filelists/train.txt

# Training

In [ ]:
#@title  Model saving/pretrained model preferences
Clone = "44k"
%cd /content/so-vits-svc

#@markdown **Save the trained model files to Google Drive instead of the colab runtime filesystem. You also need to check and execute this when resuming training. If you are short on Drive space you may want to uncheck this (in which case you must use the file menu to manually download the model checkpoints in so-vits-svc/logs/ yourself and copy the latest checkpoint back into so-vits-svc/logs/ to resume training).**
Save_to_drive = False #@param {type:"boolean"}
if Save_to_drive:
  !rm -rf /content/so-vits-svc/logs/"{Clone}"
  !mkdir -p /content/drive/MyDrive/"{Clone}"
  !ln -s /content/drive/MyDrive/"{Clone}" /content/so-vits-svc/logs/"{Clone}"

#@markdown **Download the pre-trained model for the first training. For continuing training, use the checkpoints saved by yourself after continuing to train.**
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  !wget -O logs/"{Clone}"/G_0.pth -P logs/"{Clone}"/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/G_0.pth
  !wget -O logs/"{Clone}"/D_0.pth -P logs/"{Clone}"/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/D_0.pth


In [ ]:
#@title  Start training
#@markdown **Start training**
Clone = "44k"

#@markdown **Enable tensorboard for data visualization**
tensorboard_on = True #@param {type:"boolean"}
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/"{Clone}"

!python train.py -c configs/config.json -m "{Clone}"


In [ ]:
#@title Train cluster model
!python cluster/train_cluster.py

In [ ]:
#@title Convenience cell for manual backup to Google Drive
#@markdown This cell allows for manually saving models to Google Drive (rather than your local filesystem). The code for this cell should be modified to point to the desired checkpoints to save to Google Drive.
!mv /content/so-vits-svc/logs/44k/G_1000.pth /content/drive/MyDrive
!mv /content/so-vits-svc/logs/44k/D_1000.pth /content/drive/MyDrive